In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#house price prediction in boston

Pytorch의 MLP를 이용한 집값예측(regression)구현  
  
주요 모듈 목록  
**torch.utils.data.Dataset:** 데이터셋을 getitem method에서 한개씩 텐서형태로 반환하도록 구성하는 class  
**torch.utils.data.DataLoader:** batch processing을 위해 dataset에서 반환되는 데이터를 n개의 batch_size만큼 묶어서 반환하도록 하는 class  
**torch.nn.Module:** 딥러닝 모델을 구현한 class, forward method를 통해 입력된 텐서 데이터를 딥러닝 연산하여 결과 반환  
**torch.nn.(loss):** loss 연산을 위한 class이며 backward method를 통해 gradient계산  
**torch.optim.(optimizer):** 계산된 gradient를 update해주어 feadient descent를 진행하는 class  

Dataset class구현  
  
dataset(Housingdata.csv) : https://www.kaggle.com/datasets/altavish/boston-housing-dataset

traget: MEDV(집값)  
input feature: MEDV를 제외한 모든 값  
전체 데이터 약 500개중 400개까지 train data로 사용, 나머지는 test data  

각 data를 torch.utils.data.Dataset을 통해 하나씩 load할 수 있는 class구현  
이후 torch.utils.data.DataLoader를 통해 batch개씩 변환

torch.utils.data.Dataset의 주요 method 
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(데이터셋, 데이터경로)등을 생성  
__ len __: 해당 클래스에서 다루는 dataset의 길이를 반환하는 함수  
__ getitem __(index): index에 해당하는 데이터 하나를 tensor형태로 반환하는 함수

In [17]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch

train_data_path = "/content/drive/MyDrive/여름 NLP/2일차_배포/dataset/HousingData.csv"




class myDataset(Dataset):
  def __init__(self,dataset_df)->None:
    super().__init__()
    self.dataset_df=dataset_df
    self.inputData_df = dataset_df.drop("MEDV", axis=1)
    print(self.inputData_df)
    self.target_df = dataset_df.loc[:,["MEDV"]]
    print(self.target_df)
    
  def __len__(self):
    return len(self.target_df)

  def __getitem__(self,index):
    data_tensor = torch.Tensor(self.inputData_df.loc[index,:])
    target_tensor = torch.Tensor(self.target_df.loc[index,:])
    
    return data_tensor, target_tensor

batch_size = 4

data_df = pd.read_csv(train_data_path).dropna()
train_df = data_df.loc[:400,:].reset_index()
dev_df = data_df.loc[400:,:].reset_index()

train_dataset = myDataset(train_df)
dev_dataset = myDataset(dev_df)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size)



for i in train_dataset:
  print(i)
  break

    

     index      CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD  \
0        0   0.00632  18.0   2.31   0.0  0.538  6.575   65.2  4.0900    1   
1        1   0.02731   0.0   7.07   0.0  0.469  6.421   78.9  4.9671    2   
2        2   0.02729   0.0   7.07   0.0  0.469  7.185   61.1  4.9671    2   
3        3   0.03237   0.0   2.18   0.0  0.458  6.998   45.8  6.0622    3   
4        5   0.02985   0.0   2.18   0.0  0.458  6.430   58.7  6.0622    3   
..     ...       ...   ...    ...   ...    ...    ...    ...     ...  ...   
308    396   5.87205   0.0  18.10   0.0  0.693  6.405   96.0  1.6768   24   
309    397   7.67202   0.0  18.10   0.0  0.693  5.747   98.9  1.6334   24   
310    398  38.35180   0.0  18.10   0.0  0.693  5.453  100.0  1.4896   24   
311    399   9.91655   0.0  18.10   0.0  0.693  5.852   77.8  1.5004   24   
312    400  25.04610   0.0  18.10   0.0  0.693  5.987  100.0  1.5888   24   

     TAX  PTRATIO       B  LSTAT  
0    296     15.3  396.90   4.98  
1    

Deep learning 모델 구현  
torch.nn.Module을 이용하여 모델 구현  
1st hidden layer의 feature는 100개  
2nd hidden layer의 feature는 10개  
인 모델을 구현한다.  

torch.nn.Linear: perceptron의 weighted sum과 같이 linaer regression연산을 하는 calss  
torch.nn.ReLU: ReLU activation function을 수행하는 class  

torch.nn.Module의 주요 method  
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(사용할 deep learning layer, activation function, 등)등을 생성  
__ forward __(data): 입력받은 data를 딥러닝 모델을 통해 결과를 예측하여 반환하는 class  

**각 딥러닝 레이어 연산 중 차원수를 확인하고 잘 맞춰줄 것**  
참고 document(해당 사이트의 shape를 확인하고 tensor형태 결정)  
nn.Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html  
nn.ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [18]:
from torch import nn

class myModel(nn.Module):
  def __init__(self)->None:
    super().__init__()

    self.ln1 = nn.Linear(14,100,bias=True)
    self.act1 = nn.ReLU()

    self.ln2 = nn.Linear(100,10,bias=True)
    self.ln3 = nn.Linear(10,1,bias=True)

  def forward(self,x):
    #2- , 14
    x = self.ln1(x)

    x = self.act1(x)

    x = self.ln2(x)
    x = self.act1(x)

    x = self.ln3(x)
    x = self.act1(x)

    return x

model = myModel()

for i in train_dataloader:
  data = i[0]
  target = i[1]
  output = model(data)
  print(output)
  break





tensor([[3.1446],
        [1.4531],
        [1.6547],
        [1.4930]], grad_fn=<ReluBackward0>)


작성한 dataset과 model을 이용하여 딥러닝 프로세스 구현  

pytorch 딥러닝 프로세스
1. dataset, model선언
2. dataset과 model을 통한 결과 예측
3. 예측된 결과를 통해 **loss**연산 및 **loss.backward**
4. **optimizer.step()**를 사용하여 graident update

주요 오브젝트  
torch.nn.MSELoss: 예측값과 정답을 통해 MSE값을 반환하는 class  
torch.optim.Adam: Adam optimizer를 통해 gradient update를 수행하는 class

각 오브젝트의 입력과 선언은 다음 doc 참조:  
MSELoss: https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html  
Adam: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

In [19]:
from torch.optim import Adam
from torch.nn import MSELoss

optimizer = Adam(model.parameters(),lr=0.001)
lf = MSELoss()


for e in range(100):
  print("epoch",e)
  epoch_loss = 0

  model.train()
  for i in train_dataloader:
    optimizer.zero_grad()
    data = i[0]
    target = i[1]
    pred = model(data)

    loss = lf(pred, target)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item();
    
  print("train loss", epoch_loss/len(train_dataloader))

  model.eval()
  test_loss = 0

  with torch.no_grad():

    for i in dev_dataloader:
      data = i[0]
      target = i[1]

      output = model(data)

      loss = lf(output, target)
      test_loss += loss.item()

  print("test loss",test_loss/len(dev_dataloader))

  




epoch 0
train loss 100.9456174494345
test loss 31.16543820358458
epoch 1
train loss 95.56837154038345
test loss 47.18920201346988
epoch 2
train loss 69.70888612730593
test loss 71.13887521198818
epoch 3
train loss 75.33320985262907
test loss 72.45257132393974
epoch 4
train loss 88.81744755597055
test loss 72.97620786939349
epoch 5
train loss 91.50338554080528
test loss 72.33147921448662
epoch 6
train loss 91.1757952518101
test loss 68.78010970070248
epoch 7
train loss 94.8584064109416
test loss 77.09649276733398
epoch 8
train loss 93.74874730955196
test loss 76.67063856124878
epoch 9
train loss 91.49400589435915
test loss 80.51127174922398
epoch 10
train loss 88.24666265634042
test loss 79.39382639385406
epoch 11
train loss 80.10569382619254
test loss 86.72760622841972
epoch 12
train loss 72.90860083887848
test loss 79.98947198050362
epoch 13
train loss 65.82324831697005
test loss 84.07980519249325
epoch 14
train loss 58.78160749055162
test loss 78.57248160952614
epoch 15
train loss 53